In [5]:
# 라이브러리를 불러옵니다.
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from matplotlib.colors import LogNorm
from matplotlib import rc
import seaborn as sns

import networkx as nx

In [48]:
cd data

c:\Users\ylee1\OneDrive\바탕 화면\새 폴더\2024 2학기\소넷\프로젝트\Drug_Coprescription_Network_Analysis\data


In [49]:
pd.read_csv('mapping.csv')

,주성분코드_9,ATC코드,주성분명,주성분코드_5
0,100101APD,L03AX,Other immunostimulants,1001A
1,100102ACH,L03AX,Other immunostimulants,1001A
2,100102ATB,L03AX,Other immunostimulants,1001A
3,100130ALQ,L03AX,Other immunostimulants,1001A
4,100301ACH,A05AA,Bile acids and derivatives,1003A
...,...,...,...,...
6077,801400CLQ,B02BC30,combinations,8014C
6078,801500BIJ,B05BA10,combinations,8015B
6079,801601BIJ,J01MA14,moxifloxacin,8016B
6080,920002CEX,V04CL,Tests for allergic diseases,9200C


In [ ]:
df

,기준년도,가입자일련번호,처방내역일련번호,일련번호,성별코드,연령대코드(5세단위),시도코드,요양개시일자,약품일반성분명코드,1회 투약량,1일투약량,총투여일수,단가,금액,데이터 공개일자,ATC코드
0,2021,908807,640330,1,2,13,41,2021-06-28,621901ACR,1.0,1,3,584.0,1752,2022-08-11,A02BC06
1,2021,104703,944513,1,2,13,41,2021-02-25,111501ATB,1.0,1,60,611.0,36660,2022-08-11,C10AA05
2,2021,710952,1081625,1,2,13,41,2021-10-02,415601ATB,1.0,1,3,4140.0,12420,2022-08-11,N02CC03
3,2021,856296,1707917,1,2,13,41,2021-09-28,495501ACE,1.0,1,3,363.0,1089,2022-08-11,N06AX21
4,2021,908807,2043127,1,2,13,41,2021-06-26,111001ATE,1.0,1,3,77.0,231,2022-08-11,B01AC06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1122911,2021,750125,10934119,2,2,11,11,2021-02-11,207001ATB,0.5,2,42,188.0,7896,2022-08-11,G04BD04
1122912,2021,543280,10970109,4,2,10,41,2021-02-14,495401ACH,2.0,1,28,276.0,15456,2022-08-11,C05CX
1122913,2021,321849,11006121,5,2,18,41,2021-08-01,378601ATB,1.5,1,30,265.0,11925,2022-08-11,N05AH04
1122914,2021,766500,11702317,3,1,17,44,2021-05-05,227001ATB,0.5,1,21,499.0,5240,2022-08-11,N06AB06


### 그래프 생성용 함수

In [1]:
def create_graph(data):
    G = nx.Graph()
    
    # 노드를 추가합니다.
    for atc_code in data['ATC_code'].unique():
        G.add_node(atc_code)
        
    # user_id와 pres_id로 그룹화하여 동시 발생을 찾습니다.
    user_cooccurrences = {}
    for user_id, user_group in data.groupby('user_id'):
        # 각 user_id에 대해 co-occurrence를 집합으로 저장합니다.
        cooccurrences = set()
        for pres_id, pres_group in user_group.groupby('pres_id'):
            atc_codes = pres_group['ATC_code'].unique()
            for i in range(len(atc_codes)):
                for j in range(i + 1, len(atc_codes)):
                    cooccurrence = tuple(sorted((atc_codes[i], atc_codes[j])))
                    cooccurrences.add(cooccurrence)
        user_cooccurrences[user_id] = cooccurrences

    # co-occurrence를 그래프에 추가합니다.
    for cooccurrences in user_cooccurrences.values():
        for atc_code1, atc_code2 in cooccurrences:
            if G.has_edge(atc_code1, atc_code2):
                G[atc_code1][atc_code2]['weight'] += 1
            else:
                G.add_edge(atc_code1, atc_code2, weight=1)

    return G

만성질환 목록 및 매핑 실행

In [6]:
# 만성질환 목록을 불러옵니다.
comor_mapping = pd.read_csv('data/mapping/comorbidities_mapping.csv', encoding='utf-8')

# ATC 코드를 약품명으로 바꿔줍니다.
atc_to_name = comor_mapping.set_index('ATC코드')['주성분명'].to_dict()

# 만성질환 질병 종류를 구분해줍니다. 
class_map = comor_mapping[['ATC코드', 'comorbidity_class']].drop_duplicates(subset=['ATC코드'], keep='first')
atc_to_class = class_map.set_index('ATC코드')['comorbidity_class'].to_dict()

# 만성질환 질병 종류를 구분해줍니다. (여러 질병이 하나의 ATC 코드에 매핑되는 경우)
comor_mapping['comorbidity_classes'] = comor_mapping.groupby('ATC코드')['comorbidity_class'].transform(lambda x: ' '.join(x.unique()))
atc_to_classes = comor_mapping.set_index('ATC코드')['comorbidity_classes'].to_dict()

In [7]:
temp = comor_mapping.groupby(['ATC코드'])['comorbidity_class'].nunique()
temp = temp[temp > 1]
temp

ATC코드
C03AA03    2
C03CA      2
C03CA01    2
C03CA04    2
C07AB02    2
          ..
R03DA04    2
R03DA05    2
R03DC01    2
R03DC03    2
R03DX05    2
Name: comorbidity_class, Length: 71, dtype: int64

### 컬럼명 변경용 딕셔너리

In [8]:
# 컬럼 이름들을 영문으로 바꿔줍니다
cols_rename = {
    '기준년도': 'year',
    '가입자일련번호': 'user_id',
    '처방내역일련번호': 'pres_id',
    '일련번호': 'med_id',
    '성별코드': 'gender',
    '연령대코드(5세단위)': 'age_group',
    '시도코드': 'region',
    '요양개시일자': 'start_date',
    '약품일반성분명코드': 'mi_code',
    '1회 투약량': 'dose_per_time',
    '1일투약량': 'dose_per_day',
    '총투여일수': 'total_days',
    '단가': 'unit_price',
    '금액': 'total_price',
    '데이터 공개일자': 'data_release_date',
    'ATC코드': 'ATC_code'
}

## 20-39 세 연령대의 항우울제 처방 환자 약물 네트워크 분석

In [10]:
pwd

'c:\\Users\\ylee1\\OneDrive\\바탕 화면\\새 폴더\\2024 2학기\\소넷\\프로젝트\\Drug_Coprescription_Network_Analysis'

In [11]:
# 데이터 불러오기 
df = pd.read_csv('./data/prescription_with_ad_2021.csv')

In [ ]:
# 20-39 세 환자만 뽑아오기
youngToMid = df[(df['연령대코드(5세단위)']>=5) & (df['연령대코드(5세단위)']< 9)]


In [23]:
# 컬럼 이름들을 영문으로 바꿔줍니다. 
youngToMid.rename(columns=cols_rename, inplace=True)
youngToMid

C:\Users\ylee1\AppData\Local\Temp\ipykernel_31032\82259496.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  youngToMid.rename(columns=cols_rename, inplace=True)


,year,user_id,pres_id,med_id,gender,age_group,region,start_date,mi_code,dose_per_time,dose_per_day,total_days,unit_price,total_price,data_release_date,ATC_code
776,2021,65565,720777,1,1,6,41,2021-08-23,149204ATB,1.00,1,21,110.0,2310,2022-08-11,N06AA12
786,2021,745538,1080751,1,1,6,41,2021-05-08,675801ATB,1.00,1,7,1300.0,9100,2022-08-11,A02BC09
794,2021,585410,1702981,1,2,6,41,2021-11-09,203401ATB,1.00,1,30,49.0,1470,2022-08-11,N06AA10
801,2021,203439,132686,1,2,8,41,2021-10-12,203402ATB,1.00,1,30,70.0,2100,2022-08-11,N06AA10
802,2021,487678,407826,1,2,8,41,2021-08-17,161501ACH,1.00,1,7,210.0,1470,2022-08-11,N06AB03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1122882,2021,391146,12050423,2,2,6,11,2021-08-07,204001ATB,0.25,1,7,1679.0,2938,2022-08-11,N05AH03
1122892,2021,834209,10431915,1,2,6,26,2021-07-31,113501ATB,0.50,3,14,123.0,2583,2022-08-11,M03BX01
1122893,2021,391146,11007158,2,2,6,11,2021-07-31,204001ATB,0.25,1,7,1679.0,2938,2022-08-11,N05AH03
1122902,2021,362319,8537598,1,1,8,41,2021-05-05,161502ACH,3.00,1,30,483.0,43470,2022-08-11,N06AB03


## 만성질환 ATC 코드만 데이터 프레임에서 추출 

In [30]:
comor_ATC_codes = comor_mapping['ATC코드'].unique()
comor_ATC_codes

array(['R03AC13', 'R03AC16', 'R03AC02', 'R03AC18', 'R03AK11', 'R03AK06',
       'R03AK07', 'R03AK08', 'R03AK10', 'R03AK12', 'R03AK14', 'R03BB01',
       'R03BB04', 'R03BB05', 'R03BB06', 'R03BB07', 'R03BA02', 'R03BA05',
       'R03BA08', 'R03BA09', 'R03AL03', 'R03AL05', 'R03AL06', 'R03AL08',
       'R03AL09', 'R03AL12', 'R03AL04', 'R03DA', 'R03DA05', 'R03DA08',
       'R03DA04', 'R03DA11', 'R03DA07', 'R03DC02', 'R03DC01', 'R03DC03',
       'R03DC', 'R03DC53', 'R03DX', 'R03DX05', 'R03DX07', 'R03DX09',
       'R03DX08', 'R03DX10', 'A10AB01', 'A10AB04', 'A10AB05', 'A10AC01',
       'A10AD01', 'A10AD04', 'A10AD05', 'A10AE04', 'A10AE05', 'A10BA02',
       'A10BB01', 'A10BB07', 'A10BB09', 'A10BB12', 'A10BD02', 'A10BD07',
       'A10BD15', 'A10BF01', 'A10BF02', 'A10BG03', 'A10BH01', 'A10BH02',
       'A10BH05', 'A10BJ01', 'A10BK01', 'A10BK03', 'A10BX02', 'A10BX03',
       'C03AA03', 'C03BA11', 'C03BA10', 'C03BA08', 'C03BA04', 'C03CA',
       'C03CA01', 'C03CA04', 'C03DA01', 'C03DA05', 'C03DB01

In [31]:
young_df = youngToMid[youngToMid['ATC_code'].isin(comor_ATC_codes)]
# 환자수 추출. 
young_df['user_id'].nunique()

1961

In [45]:
young_df['user_id'].nunique()

1961

## 20~39세 그래프 생성 

In [32]:
# 그래프를 생성합니다.
G_young = create_graph(young_df)

# 그래프의 노드와 엣지 수를 출력합니다.
print(f"Number of nodes: {G_young.number_of_nodes()}")
print(f"Number of edges: {G_young.number_of_edges()}")

Number of nodes: 93
Number of edges: 190


In [33]:
# 그래프에 label을 추가합니다.
for node in G_young.nodes:
    G_young.nodes[node]['label'] = atc_to_name[node]
    G_young.nodes[node]['comorbidity'] = atc_to_class[node]
    G_young.nodes[node]['comorbidities'] = atc_to_classes[node]

# 게피파일로 저장합니다

In [35]:
pwd

'c:\\Users\\ylee1\\OneDrive\\바탕 화면\\새 폴더\\2024 2학기\\소넷\\프로젝트\\Drug_Coprescription_Network_Analysis'

In [38]:
cd data

c:\Users\ylee1\OneDrive\바탕 화면\새 폴더\2024 2학기\소넷\프로젝트\Drug_Coprescription_Network_Analysis\data


In [42]:
cd ..

c:\Users\ylee1\OneDrive\바탕 화면\새 폴더\2024 2학기\소넷\프로젝트\Drug_Coprescription_Network_Analysis


In [43]:
nx.write_gexf(G_young, 'gephi/depression_young_age_prescription_network.gexf')